# Hass Client

Create a long lived access token https://www.home-assistant.io/docs/authentication/

In [1]:
import secrets

api_token = secrets.get_secret("hass_token")
hostport = secrets.get_secret("hostport")

In [4]:

from hass_data_client import create_client

client = await create_client(hostport, api_token)

In [5]:
states = await client.get_states()
len(states)

686

# Statistics

In [4]:
import datetime 

now = datetime.datetime.now()

start = now - datetime.timedelta(days=30)
end = start + datetime.timedelta(days=1)

result = await client.send_command(
    "recorder/statistics_during_period",
    start_time=start.isoformat(),
    end_time=end.isoformat(),
    period="hour",
    statistic_ids=["sensor.weather_temperature"],
    types=["mean", "state"],
)


In [5]:
result

{'sensor.weather_temperature': [{'start': 1703815200000,
   'end': 1703818800000,
   'mean': 54.928756172777774},
  {'start': 1703818800000, 'end': 1703822400000, 'mean': 53.531315820555555},
  {'start': 1703822400000, 'end': 1703826000000, 'mean': 53.40201823638889},
  {'start': 1703826000000, 'end': 1703829600000, 'mean': 53.664695450833335},
  {'start': 1703829600000, 'end': 1703833200000, 'mean': 52.46316521833334},
  {'start': 1703833200000, 'end': 1703836800000, 'mean': 51.0},
  {'start': 1703836800000, 'end': 1703840400000, 'mean': 51.0},
  {'start': 1703840400000, 'end': 1703844000000, 'mean': 50.931589800000005},
  {'start': 1703844000000, 'end': 1703847600000, 'mean': 50.0},
  {'start': 1703847600000, 'end': 1703851200000, 'mean': 50.0},
  {'start': 1703851200000, 'end': 1703854800000, 'mean': 49.06491863444444},
  {'start': 1703854800000, 'end': 1703858400000, 'mean': 49.0},
  {'start': 1703858400000, 'end': 1703862000000, 'mean': 48.19823716222222},
  {'start': 170386200000

In [6]:
len(result['sensor.weather_temperature'])

24

# Entities

Query available entities.

In [7]:
result = await client.send_command(
    "config/entity_registry/list",
)
len([ r['entity_id'] for r in result ])

1274

In [8]:
result = await client.send_command(
    "config/entity_registry/list_for_display",
)
entity_ids = [ e['ei'] for e in result['entities'] ]
len(entity_ids)

682

# Services

In [10]:
result = await client.send_command("get_services")

In [11]:
service_names = [
    f"{domain}.{name}"
    for domain, services in result.items()
    for name in services
]
print(f"Domains: {len(result.items())}")
print(f"Services: {len(service_names)}")

Domains: 71
Services: 278


In [12]:
result['todo']['get_items']

{'name': 'Get to-do list items',
 'description': 'Get items on a to-do list.',
 'fields': {'status': {'example': 'needs_action',
   'default': 'needs_action',
   'selector': {'select': {'translation_key': 'status',
     'options': ['needs_action', 'completed'],
     'multiple': True}},
   'name': 'Status',
   'description': 'Only return to-do items with the specified statuses. Returns not completed actions by default.'}},
 'target': {'entity': [{'domain': ['todo']}]},
 'response': {'optional': False}}

In [13]:
[ entity_id for entity_id in entity_ids if entity_id.startswith("todo") ]

['todo.maintenance', 'todo.grocery', 'todo.my_tasks']

In [14]:
result = await client.send_command(
    "call_service",
    domain="todo",
    service="get_items",
    target={"entity_id": "todo.my_tasks"},
    return_response=True,
)

In [15]:
result['response']['todo.my_tasks']['items'][0]

{'summary': 'Med prompt',
 'uid': 'UHh2aVVTTVUzenVsWmFqWA',
 'status': 'needs_action'}

# Events

The hass client needs a streaming API to be able to return these messages.

In [16]:
import logging

logging.getLogger().setLevel(logging.DEBUG)


In [17]:
import datetime 

now = datetime.datetime.now()

start = now + datetime.timedelta(days=-1)
end = start + datetime.timedelta(days=1)

result = await client.send_command_no_wait(
    "logbook/event_stream",
    start_time=start.isoformat(),
    end_time=end.isoformat(),
)
result
